In [1]:
from azureml.core import Workspace, Experiment

# Configure workspace and experiment
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="titanic-optimization")

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# cluster configuration
cluster_name = "mldemocompute"
min_nodes = 0
max_nodes = 4
vm_size = "STANDARD_D2_V2"

# verify that the cluster exists already
try:
    aml_cluster = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    print('Cluster not `%s` not found, creating one now.' % cluster_name)
    config = AmlCompute.provisioning_configuration(vm_size=vm_size, min_nodes=min_nodes, max_nodes=max_nodes)
    aml_cluster = ComputeTarget.create(workspace=ws, name=cluster_name, provisioning_configuration=config)

# wait until the cluster is ready
aml_cluster.wait_for_completion(show_output=True)

Cluster not `mldemocompute` not found, creating one now.
InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [33]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
 
def get_env(packages=None):
    packages = packages or []
    packages += ['azureml-defaults']
    env = Environment(name="tf")
    env.python.conda_dependencies = CondaDependencies.create(pip_packages=packages)
    return env

In [34]:
tf_env = get_env(['numpy', 'pandas', 'scikit-learn', 'tensorflow'])

In [35]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory="train", 
                      script="train.py",
                      compute_target = aml_cluster,
                      environment = tf_env)
run = exp.submit(src)
print(run.get_portal_url())

https://ml.azure.com/runs/titanic-optimization_1636413082_0b95dbda?wsid=/subscriptions/21dc412b-d9eb-42e7-8317-55bc8eb10cf5/resourcegroups/packt/workspaces/mldemows&tid=e702ab66-1ac1-42ba-b41b-8195d943a1e0


In [36]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…